In [19]:
import numpy as np
import pandas as pd
import sklearn
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

In [26]:
batch_size=64
#PIL to Tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

train_dataset = datasets.MNIST("./data",train=True,transform=transform,download=True)
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset = datasets.MNIST("./data",train=False,transform=transform,download=True)


In [27]:
train_loader=DataLoader(dataset=train_dataset,batch_size=32,shuffle=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=32,shuffle=False)

In [22]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        
        self.l1 = torch.nn.Linear(784,512,True) # input output
        self.l2 = torch.nn.Linear(512,156,True) # input output
        self.l3 = torch.nn.Linear(256,128,True) # input output
        self.l4 = torch.nn.Linear(128,64,True) # input output
        self.l5 = torch.nn.Linear(64,10,True) # input output
#         self.sigmoid=torch.nn.Sigmoid()
        
        
    def forward(self,x):
        """__call__ 中被调用  """
        x=x.view(-1,784)
        x=F.relu(self.l1(x))
        x=F.relu(self.l2(x))
        x=F.relu(self.l3(x))
        x=F.relu(self.l4(x))
        return self.l5(x)
    
model = Model()

criterion =torch.nn.CrossEntropyLoss() 
optimizer=torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.5)

In [23]:
def train(epoch):
    running_loss=0.0
    for batch_idx,data in enumerate(train_loader,0):
        inputs,labels=data
        optimizer.zero_grad()
        outputs=model(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        running_loss +=  loss.item()
        if batch_idx % 300== 299:
            print('[%d,%5d] loss: %.3f' % (epoch+1,batch_idx+1,running_loss/300))
            running_loss=0.0

In [24]:
def test():
    correct=0
    total=0
    with torch.no_grad():
        for data in test_loader:
            images,labels= data
            outputs=model(images)
            _,predicted=torch.max(outputs.data,dim=1)
            total += labels.size(0)
            correct+=(predicted == labels).sum().item()
    print("accuracy :", (100*correct)/total)

In [25]:
for epoch in range(10):
    train(epoch)
    test()

RuntimeError: size mismatch, m1: [32 x 156], m2: [256 x 128] at /Users/distiller/project/conda/conda-bld/pytorch_1591914925853/work/aten/src/TH/generic/THTensorMath.cpp:41